In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [3]:
code = []
title = []
intro = []

cookies = {
    'ACEUCI': '1',
    'CheckSameSite': 'IsValidSameSiteSet',
    'AladdinUser': 'UID=-50125363&SID=4P5oPL5cRydu48l0%2b6Q1uQ%3d%3d',
    'AladdinSession': 'UID=-50125363&SID=4P5oPL5cRydu48l0%2b6Q1uQ%3d%3d',
    'AladdinUS': '8VIA9uXIEEDg0PBcYe2Wbg%3d%3d&USA=0',
    'ACEUACS': '1597764680612231040',
    'ACEFCID': 'UID-5F4875F0E779D63E9098338F',
    '_BS_GUUID': 'dNt4ok3JKAKZLWQCygDAdQYK4H7EDqBRd4VCpZla',
    '_ga': 'GA1.3.1224188238.1598584305',
    '_gid': 'GA1.3.645720543.1598584305',
    'ala_qs_use': '1',
    'pFloatingBanner': 'true',
    '_TRK_AUIDA_13987': '4862220279678af7bef6c760c9955607:2',
    '_TRK_ASID_13987': 'd44039508014a1158acc584f60cc1c04',
    'refererURL': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=246064543',
    '_gat_gtag_UA_59266_2': '1',
}

headers = {
    'Connection': 'keep-alive',
    'Accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=246064543',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6',
}

for i in range(1, 21): #21
    response = requests.get(f'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=351&page={i}&cnt=1000&SortOrder=1')
    soup = BeautifulSoup(response.text, 'html.parser')
    div_tag = soup.select('form#Myform > div.ss_book_box')
    print(i, end="")
    for div in div_tag:
        a_tag = div.select_one('li > a.bo3')
        title.append(a_tag.get_text()) # 책 제목
        
        # isbn
        src=div.select_one('img.i_cover')['src']
        isbn = (src.split("cover150/")[1]).split("_")[0]
        
        a_href = a_tag['href']
        code.append(a_href.split("ItemId=")[1]) # 코드
        
        params = (
            ('ISBN', isbn),
            ('name', 'Introduce'),
            ('type', '0'),
            ('date', '14'),
        )

        response = requests.get('https://www.aladin.co.kr/shop/product/getContents.aspx', headers=headers, params=params, cookies=cookies)
        soup = BeautifulSoup(response.text, 'html.parser')
        ls = soup.select_one('div.Ere_prod_mconts_LS')
        if (ls is not None) and (ls.get_text()=='책소개'):
            div_r = soup.select_one('div.Ere_prod_mconts_R')
            if div_r is not None:
                text = div_r.get_text()
                text = text.replace('\r\n','').replace('  ','').replace('\n','')
                intro.append(text)
            elif div_r is None:
                intro.append("NULL")
        else:
            intro.append("NULL")

1234567891011121314151617181920

In [4]:
dic = {
    'code':code,
    'title':title,
    'intro':intro
}

In [5]:
import pandas as pd
df=pd.DataFrame(dic)
print(df.shape)
df.head()

(1000, 3)


,code,title,intro
0,246064543,비전공자를 위한 이해할 수 있는 IT 지식,"비전공자를 위한 'IT 기초 문법서'로, 한 번쯤 들어봤을 프로그래밍 언어, 운영체..."
1,193931483,혼자 공부하는 파이썬,"독학으로 프로그래밍 언어를 처음 배우려는 입문자가, 혹은 파이썬을 배우려는 입문자가..."
2,236186172,리팩터링 2판 (리팩토링 개정판),"개발자가 선택한 프로그램 가치를 높이는 최고의 코드 관리 기술, 마틴 파울러의 『리..."
3,253657487,최고의 그림을 그리는 방법,베스트셀러 <DVD와 함께하는 애니메이션 캐릭터 작화 기술>의 저자가 경험을 바탕으...
4,99386523,직장인을 위한 실무 엑셀,NULL
